# Load data

In [14]:
import pandas as pd

In [15]:
from pathlib import Path

def load_data(path: str) -> pd.DataFrame:
    data = pd.read_csv(Path(path))
    return data

In [16]:
data = load_data("../data/data.csv")

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


# Preprocessing

In [19]:
data_preprocessed = pd.DataFrame()

# Split data

In [ ]:
input_features = [col for col in data.columns if col != 'target']
X = data[input_features]
y = data['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Training

## K-NN

## Linear regression

## Ridge regression

## Lasso regression

## BERT

# Evaluation

In [ ]:
x0 = [0 for _ in range(len(embeddings[0]))]

In [ ]:
for embedding in embeddings:
    data.add_column(embedding)
    for i, component in enumerate(embedding):
        data[f"x_{i}"] = component